# Trained ResNet-50 classification

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import datetime
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
# Project
import sys
sys.path.append(os.path.join(os.path.abspath(os.path.dirname('.')), '..', 'common'))
from data_utils import type_1_ids, type_2_ids, type_3_ids, test_ids
from training_utils import get_trainval_id_type_lists, get_test_id_type_list, data_iterator, data_augmentation
from metrics import logloss_mc

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GT 750M (CNMeM is enabled with initial size: 50.0% of memory, cuDNN 5103)


In [4]:
print("\n=========================")
print("Training dataset: ")
print("- type 1: ", len(type_1_ids))
print("- type 2: ", len(type_2_ids))
print("- type 3: ", len(type_3_ids))

print("Test dataset: ")
print("- ", len(test_ids))
print("=========================\n")



Training dataset: 
('- type 1: ', 250)
('- type 2: ', 781)
('- type 3: ', 450)
Test dataset: 
('- ', 512)



In [23]:
def train(model, train_id_type_list, val_id_type_list, batch_size=16, nb_epochs=10, image_size=(224, 224)):
    
    samples_per_epoch = (2048 // batch_size) * batch_size
    nb_val_samples = (1024 // batch_size) * batch_size

    if not os.path.exists('weights'):
        os.mkdir('weights')

    weights_filename = os.path.join("weights", "resnet_simple_{epoch:02d}-{val_loss:.4f}.h5")
    model_checkpoint = ModelCheckpoint(weights_filename, monitor='loss', save_best_only=True)

    print("Training parameters: ", batch_size, nb_epochs, samples_per_epoch, nb_val_samples)
    
    train_iter = data_iterator(train_id_type_list, 
                               batch_size=batch_size, 
                               image_size=image_size, 
                               data_augmentation_fn=lambda X, Y: data_augmentation(X, Y),
                               verbose=0)
    val_iter = data_iterator(val_id_type_list, batch_size=batch_size, image_size=image_size, verbose=0)
    
    history = model.fit_generator(
        train_iter,
        samples_per_epoch=samples_per_epoch,
        nb_epoch=nb_epochs,
        validation_data=val_iter,
        nb_val_samples=nb_val_samples,
        callbacks=[model_checkpoint],
        verbose=1,
    )

    return history


def validate(model, val_id_type_list, batch_size=16, image_size=(224, 224)):
    val_iter = data_iterator(val_id_type_list, batch_size=batch_size, image_size=image_size, test_mode=True)

    total_loss = 0.0
    total_counter = 0 
    for X, Y_true, _ in val_iter:           
        s = Y_true.shape[0]
        total_counter += s
        Y_pred = model.predict(X)
        loss = logloss_mc(Y_true, Y_pred)
        total_loss += s * loss
        print("--", total_counter, "batch loss : ", loss)

    if total_counter == 0:
        total_counter += 1

    total_loss *= 1.0 / total_counter   
    print("Total loss : ", total_loss)
    
    
def predict(model, batch_size=16, image_size=(224, 224), info=''):

    test_id_type_list = get_test_id_type_list()
    test_iter = data_iterator(test_id_type_list, batch_size=batch_size, image_size=image_size, test_mode=True)
    
    df = pd.DataFrame(columns=['image_name','Type_1','Type_2','Type_3'])
    total_counter = 0
    for X, _, image_ids in test_iter:            
        Y_pred = model.predict(X)    
        s = X.shape[0]
        print("--", total_counter)
        for i in range(s):
            df.loc[total_counter, :] = (image_ids[i] + '.jpg', ) + tuple(Y_pred[i, :])
            total_counter += 1

    now = datetime.datetime.now()
    sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    sub_file = os.path.join('..', 'results', sub_file)
    df.to_csv(sub_file, index=False)
    
    !cat {sub_file} | wc -l

In [6]:
from resnet_keras122 import get_resnet50
from keras.callbacks import ModelCheckpoint
from keras import __version__
print("Keras version: ", __version__)

('Keras version: ', '1.2.2')


In [7]:
print("\n {} - Get train/val lists ...".format(datetime.datetime.now()))
train_id_type_list, val_id_type_list = get_trainval_id_type_lists()


 2017-03-20 00:06:25.939047 - Get train/val lists ...
Train dataset contains : 
('-', [175, 546, 315], ' images of corresponding types')
Validation dataset contains : 
('-', [75, 234, 135], ' images of corresponding types')


In [8]:
print("\n {} - Get ResNet-50 model ...".format(datetime.datetime.now()))
resnet = get_resnet50()
resnet.summary()


 2017-03-20 00:06:25.959272 - Get ResNet-50 model ...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 230, 230)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, 112, 112)  9472        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, 112, 112)  256         conv1[0][0]                      
____________________________________

In [29]:
nb_epochs = 50
batch_size = 10

print("\n {} - Start training ...".format(datetime.datetime.now()))
history = train(resnet, train_id_type_list, val_id_type_list, nb_epochs=nb_epochs, batch_size=batch_size)


 2017-03-20 02:22:26.927029 - Start training ...
('Training parameters: ', 10, 50, 2040, 1020)
Epoch 1/50
2040/2040 [==============================] - 317s - loss: 1.0552 - acc: 0.4363 - val_loss: 1.0491 - val_acc: 0.4481
Epoch 2/50
2040/2040 [==============================] - 307s - loss: 1.0787 - acc: 0.3990 - val_loss: 1.0471 - val_acc: 0.4902
Epoch 3/50
2040/2040 [==============================] - 309s - loss: 1.0634 - acc: 0.4176 - val_loss: 1.0440 - val_acc: 0.4639
Epoch 4/50
2040/2040 [==============================] - 307s - loss: 1.0544 - acc: 0.4196 - val_loss: 1.0372 - val_acc: 0.4775
Epoch 5/50
2040/2040 [==============================] - 308s - loss: 1.0676 - acc: 0.4103 - val_loss: 1.0474 - val_acc: 0.4639
Epoch 6/50
2040/2040 [==============================] - 308s - loss: 1.0467 - acc: 0.4353 - val_loss: 1.0355 - val_acc: 0.5020
Epoch 7/50
2040/2040 [==============================] - 308s - loss: 1.0512 - acc: 0.4461 - val_loss: 1.0341 - val_acc: 0.4971
Epoch 8/50
2040

In [30]:
print("\n {} - Start validation ...".format(datetime.datetime.now()))
validate(resnet, val_id_type_list, batch_size=batch_size)


 2017-03-20 09:04:13.069935 - Start validation ...
('--', 10, 'batch loss : ', 1.5363071)
('--', 20, 'batch loss : ', 1.2154288)
('--', 30, 'batch loss : ', 1.4314357)
('--', 40, 'batch loss : ', 1.3091282)
('--', 50, 'batch loss : ', 1.7048576)
('--', 60, 'batch loss : ', 1.5230838)
('--', 70, 'batch loss : ', 1.3382587)
('--', 80, 'batch loss : ', 1.6548382)
('--', 90, 'batch loss : ', 1.7152687)
('--', 100, 'batch loss : ', 1.0297288)
('--', 110, 'batch loss : ', 1.4576861)
('--', 120, 'batch loss : ', 1.5834022)
('--', 130, 'batch loss : ', 1.3640138)
('--', 140, 'batch loss : ', 1.6267732)
('--', 150, 'batch loss : ', 1.2713295)
('--', 160, 'batch loss : ', 1.4513747)
('--', 170, 'batch loss : ', 1.5349351)
('--', 180, 'batch loss : ', 1.2707913)
('--', 190, 'batch loss : ', 1.3186457)
('--', 200, 'batch loss : ', 1.1580483)
('--', 210, 'batch loss : ', 1.4392743)
('--', 220, 'batch loss : ', 1.2794678)
('--', 230, 'batch loss : ', 1.8261249)
('--', 240, 'batch loss : ', 1.118469

In [31]:
print("\n {} - Start predictions and write submission ...".format(datetime.datetime.now()))
predict(resnet, info='resnet50_4d_no_additional', batch_size=8)


 2017-03-20 09:07:09.809985 - Start predictions and write submission ...
('--', 0)
('--', 8)
('--', 16)
('--', 24)
('--', 32)
('--', 40)
('--', 48)
('--', 56)
('--', 64)
('--', 72)
('--', 80)
('--', 88)
('--', 96)
('--', 104)
('--', 112)
('--', 120)
('--', 128)
('--', 136)
('--', 144)
('--', 152)
('--', 160)
('--', 168)
('--', 176)
('--', 184)
('--', 192)
('--', 200)
('--', 208)
('--', 216)
('--', 224)
('--', 232)
('--', 240)
('--', 248)
('--', 256)
('--', 264)
('--', 272)
('--', 280)
('--', 288)
('--', 296)
('--', 304)
('--', 312)
('--', 320)
('--', 328)
('--', 336)
('--', 344)
('--', 352)
('--', 360)
('--', 368)
('--', 376)
('--', 384)
('--', 392)
('--', 400)
('--', 408)
('--', 416)
('--', 424)
('--', 432)
('--', 440)
('--', 448)
('--', 456)
('--', 464)
('--', 472)
('--', 480)
('--', 488)
('--', 496)
('--', 504)
     513
